In [6]:
import sys
from dataclasses import dataclass
from enum import Enum, auto
from typing import *


class Keyword(Enum):
    WHILE = auto()
    WHILE_NOT = auto()
    IF = auto()
    IF_NOT = auto()
    READ = auto()
    WRITE = auto()
    COMMAND_END = auto()
    CONDITION_START = auto()
    CONDITION_END = auto()
    BLOCK_START = auto()
    BLOCK_END = auto()
    PLUS = auto()
    MINUS = auto()
    MUL = auto()
    DIV = auto()
    DEFINITION = auto()
    OPEN_PARENTHESIS = auto()
    CLOSE_PARENTHESIS = auto()
    ARRAY_DELIMITER = auto() #
    ARRAY = auto() #
    ARRAY_INDEX = auto() #


KEYWORD_BY_NAMES: Dict[str, Keyword] = {
    "while": Keyword.WHILE,
    "whilenot": Keyword.WHILE_NOT,
    "if": Keyword.IF,
    "ifnot": Keyword.IF_NOT,
    "read>": Keyword.READ,
    "write>": Keyword.WRITE,
    ";": Keyword.COMMAND_END,
    "[": Keyword.CONDITION_START,
    "]": Keyword.CONDITION_END,
    "{": Keyword.BLOCK_START,
    "}": Keyword.BLOCK_END,
    "+": Keyword.PLUS,
    "-": Keyword.MINUS,
    "*": Keyword.MUL,
    "/": Keyword.DIV,
    "=": Keyword.DEFINITION,
    "(": Keyword.OPEN_PARENTHESIS,
    ")": Keyword.CLOSE_PARENTHESIS,
    "|": Keyword.ARRAY, #
    ",": Keyword.ARRAY_DELIMITER, #
    "#": Keyword.ARRAY_INDEX, #
}

KEYWORD_NAMES: Dict[Keyword, str] = {v: k for k, v in KEYWORD_BY_NAMES.items()}

class TokenType(Enum):
    NUMBER = auto()
    WORD = auto()
    KEYWORD = auto()


# file row col
Location = Tuple[str, int, int]

@dataclass
class Token:
    typ: TokenType
    text: str
    loc: Location
    value: Union[float, str, Keyword]


class InstructionType(Enum):
    ADD = auto()
    SUB = auto()
    MUL = auto()
    DIV = auto()
    READ = auto()
    WRITE = auto()
    GOTO_IF_NOT = auto()
    GOTO_IF = auto()
    GOTO = auto()
    COPY = auto()
    ARRAY = auto() #


INSTRUCTION_BY_NAMES: Dict[str, InstructionType] = {
    "ADD": InstructionType.ADD,
    "SUB": InstructionType.SUB,
    "MUL": InstructionType.MUL,
    "DIV": InstructionType.DIV,
    "READ": InstructionType.READ,
    "WRITE": InstructionType.WRITE,
    "GOTOIFNOT": InstructionType.GOTO_IF_NOT,
    "GOTOIF": InstructionType.GOTO_IF,
    "GOTO": InstructionType.GOTO,
    "COPY": InstructionType.COPY,
    "ARRAY": InstructionType.ARRAY, #
}

INSTRUCTION_NAMES: Dict[InstructionType, str] = {v: k for k, v in INSTRUCTION_BY_NAMES.items()}

@dataclass
class Instruction:
    typ: InstructionType
    args: List[Union[float, str]]

def token_priority(token: Token) -> int:
    if token.typ != TokenType.KEYWORD:
        return -2

    v = token.value
    if v == Keyword.OPEN_PARENTHESIS or v == Keyword.CLOSE_PARENTHESIS:
        return 0
    elif v == Keyword.PLUS or v == Keyword.MINUS:
        return 1
    elif v == Keyword.MUL or v == Keyword.DIV:
        return 2
    else:
        return -1


def find_col(line: str, start: int, predicate: Callable[[str], bool]) -> int:
    while start < len(line) and not predicate(line[start]):
        start += 1
    return start


def lex_lines(file_path: str, lines: List[str]) -> Generator[Token, None, None]:
    row = 0
    while row < len(lines):
        line = lines[row]
        col = find_col(line, 0, lambda x: not x.isspace())
        col_end = col
        while col < len(line):
            if line[col].isalnum():
                col_end = find_col(line, col, lambda x: not (x.isalnum() or x == ">" or x == "." or x == "#")) #
            elif line[col] in KEYWORD_BY_NAMES:
                col_end = col + 1

            loc = (file_path, row + 1, col + 1)
            text_of_token = line[col:col_end]

            try:
                yield Token(TokenType.NUMBER, text_of_token, loc, float(text_of_token))
            except ValueError:
                if text_of_token in KEYWORD_BY_NAMES:
                    yield Token(TokenType.KEYWORD, text_of_token, loc, KEYWORD_BY_NAMES[text_of_token])
                else:
                    yield Token(TokenType.WORD, text_of_token, loc, text_of_token)

            col = find_col(line, col_end, lambda x: not x.isspace())
        row += 1


def lex_file(file_path: str) -> List[Token]:
    with open(file_path, "r", encoding='utf-8') as f:
        return [token for token in lex_lines(file_path, f.readlines())]


def compile_error(text: str):
    print("Compilation error:")
    print(text)
    sys.exit(1)


def expected(loc: Location, after: str, exp: str, found: str):
    compile_error(
        f"{loc[0]}:{loc[1]}:{loc[2]}: after {after} expected {exp}, but found {found}")


def expected_command_end(loc: Location, after: str, found: str):
    expected(loc, after, f"'{KEYWORD_NAMES[Keyword.COMMAND_END]}'", found)


def convert_to_postfix(tokens: List[Token]) -> List[Token]:
    ops = []
    res = []

    # Does not checks if tokens are correct
    for token in tokens:
        if token.typ == TokenType.WORD or token.typ == TokenType.NUMBER:
            res.append(token)
            continue

        if token.value == Keyword.OPEN_PARENTHESIS:
            ops.append(token)
        elif token.value == Keyword.CLOSE_PARENTHESIS:
            while ops[-1].value != Keyword.OPEN_PARENTHESIS:
                res.append(ops.pop())
            ops.pop()
        else:
            while len(ops) != 0 and token_priority(ops[-1]) >= token_priority(token):
                res.append(ops.pop())
            ops.append(token)

    while len(ops) != 0:
        res.append(ops.pop())

    return res


def eval_expression(tokens: List[Token], varc: int) -> Tuple[List[Instruction], str, int]:
    inst = []
    values = []

    if len(tokens) == 1:
        t = tokens[0]
        if t.typ != TokenType.WORD and t.typ != TokenType.NUMBER:
            expected(t.loc, "expression start", "variable or number", str(t.typ))
        inst.append(Instruction(InstructionType.COPY, [t.value, "tmp"]))
        return inst, f"t{varc - 1}", varc + 1

    for token in tokens:
        if token.typ == TokenType.NUMBER or token.typ == TokenType.WORD:
            values.append(token)
            continue

        right_value = values.pop().value
        left_value = values.pop().value

        var = f"t{varc}"
        varc += 1
        var_t = Token(TokenType.WORD, var, ("tmp", -1, -1), var)

        args = [left_value, right_value, var]
        if token.value == Keyword.PLUS:
            inst.append(Instruction(InstructionType.ADD, args))
            values.append(var_t)
        elif token.value == Keyword.MINUS:
            inst.append(Instruction(InstructionType.SUB, args))
            values.append(var_t)
        elif token.value == Keyword.MUL:
            inst.append(Instruction(InstructionType.MUL, args))
            values.append(var_t)
        elif token.value == Keyword.DIV:
            inst.append(Instruction(InstructionType.DIV, args))
            values.append(var_t)
        else:
            expected(token.loc, "expression part", "expression", token.text)

    return inst, f"t{varc - 1}", varc


def parse_expression(i: int, tokens: List[Token], varc: int) -> Tuple[List[Instruction], int, int]:
    expr_tokens = []

    while i < len(tokens):
        t = tokens[i]
        i += 1

        if t.typ == TokenType.KEYWORD:
            if t.value == Keyword.CONDITION_START:
                continue
            if t.value == Keyword.COMMAND_END or t.value == Keyword.CONDITION_END:
                break

        expr_tokens.append(t)

    if i > len(tokens):
        expected(tokens[i - 1].loc, "expression",
                 f"{KEYWORD_NAMES[Keyword.COMMAND_END]} or {KEYWORD_NAMES[Keyword.CONDITION_END]}",
                 f"{tokens[i - 1].text}")

    expr_tokens = convert_to_postfix(expr_tokens)
    inst, r_tmp, varc = eval_expression(expr_tokens, varc)

    return inst, i, varc - 1


def parse_condition(with_not: bool, i: int, tokens: List[Token], varc: int) -> Tuple[List[Instruction], int, int]:
    expr = parse_expression(i, tokens, varc)
    inst = expr[0]
    varc = expr[2]

    var = f"t{varc}"
    varc += 1
    inst[-1].args[-1] = var

    i = expr[1]

    if with_not:
        inst.append(Instruction(InstructionType.GOTO_IF, [var]))
    else:
        inst.append(Instruction(InstructionType.GOTO_IF_NOT, [var]))

    return inst, i, varc


def parse_program_from_tokens(tokens: List[Token]) -> List[Instruction]:
    i = 0
    varc = 1
    instructions: List[Instruction] = []
    back_stack: List[Tuple[int, Keyword]] = []
    l = len(tokens)
    while i < l:
        t = tokens[i]
        i += 1

        if t.typ == TokenType.NUMBER:
            if i >= l:
                expected_command_end(t.loc, "bare number", "nothing")
            tn = tokens[i]
            if tn.value == Keyword.COMMAND_END:
                i += 1
                continue
            expected_command_end(tn.loc, "bare number", f"'{tn.text}'")
        elif t.typ == TokenType.WORD:
            if i >= l:
                expected(t.loc, f"'{KEYWORD_NAMES[t.value]}'", f"{KEYWORD_NAMES[Keyword.DEFINITION]} or expression",
                         "nothing")
            tn = tokens[i]
            i += 1

            if tn.typ == TokenType.KEYWORD:
                if tn.value == Keyword.DEFINITION:
                    tnn = tokens[i]
                    if tnn.value == Keyword.ARRAY:
                        i += 1
                        args: List[Union[float, str]] = [t.text]

                        while tokens[i].value != Keyword.ARRAY:
                            if tokens[i].value == Keyword.ARRAY_DELIMITER:
                                i += 1
                                continue
                            args.append(tokens[i].value)
                            i += 1
                        i += 2

                        instructions.append(Instruction(InstructionType.ARRAY, args))
                    else:
                        exps = parse_expression(i, tokens, varc)
                        instructions += exps[0]
                        instructions[-1].args[-1] = t.text
                        i += exps[1] - i
                        varc += exps[2] - varc
                else:
                    if i >= l:
                        expected_command_end(t.loc, "bare variable", "nothing")
                    else:
                        tn = tokens[i]
                        i += 1
                        expected_command_end(tn.loc, "bare variable", f"'{tn.text}'")
            else:
                expected(t.loc, f"'{KEYWORD_NAMES[t.value]}'", "keyword", f"{tn.text}")
        else:
            if t.value == Keyword.IF or \
                    t.value == Keyword.WHILE or \
                    t.value == Keyword.IF_NOT or \
                    t.value == Keyword.WHILE_NOT:

                with_not = t.value == Keyword.IF_NOT or t.value == Keyword.WHILE_NOT
                cond = parse_condition(with_not, i, tokens, varc)
                instructions += cond[0]
                i += cond[1] - i
                varc += cond[2] - varc

                i += 1

                back_stack.append((len(instructions) - 1, t.value))
            elif t.value == Keyword.BLOCK_END:
                r = back_stack.pop()

                if r[1] == Keyword.IF or r[1] == Keyword.IF_NOT:
                    instructions[r[0]].args.append(str(len(instructions)))
                elif r[1] == Keyword.WHILE or r[1] == Keyword.WHILE_NOT:
                    instructions.append(Instruction(InstructionType.GOTO, [str(r[0] - 1)]))
                    instructions[r[0]].args.append(len(instructions))
                else:
                    compile_error(f"{t.loc} unsupported back keyword: {t.value}")
            elif t.value == Keyword.READ:
                if i >= l:
                    expected(t.loc, f"'{KEYWORD_NAMES[t.value]}'", "variable", "nothing")
                tn = tokens[i]
                i += 1
                if i >= l:
                    expected_command_end(tn.loc, "variable", "nothing")
                if tn.typ != TokenType.WORD:
                    expected(t.loc, f"'{KEYWORD_NAMES[t.value]}'", "variable", f"'{t.text}'")
                instructions.append(Instruction(InstructionType.READ, [tn.value]))
                i += 1
            elif t.value == Keyword.WRITE:
                if i >= l:
                    expected(t.loc, f"'{KEYWORD_NAMES[t.value]}'", "variable", "nothing")
                tn = tokens[i]
                i += 1
                if i >= l:
                    expected_command_end(tn.loc, "variable", "nothing")
                if tn.typ != TokenType.WORD and tn.typ != TokenType.NUMBER:
                    expected(t.loc, f"'{KEYWORD_NAMES[t.value]}'", "variable", f"'{t.text}'")
                instructions.append(Instruction(InstructionType.WRITE, [tn.value]))
                i += 1
            else:
                assert False, f"Parsing keyword {KEYWORD_NAMES[t.value]} not implemented"

    return instructions


def parse_program_from_file(file_path: str) -> List[Instruction]:
    tokens = lex_file(file_path)
    return parse_program_from_tokens(tokens)


def write_instruction(file_path: str, inst: List[Instruction]):
    with open(file_path, "w") as f:
        for i in inst:
            f.write(f"{INSTRUCTION_NAMES[i.typ]} ")
            f.write(" ".join([str(a) for a in i.args]))
            f.write("\n")


def is_var(text: str) -> bool:
    return not text.replace(".", "", 1).isnumeric()


def is_array(text: str) -> bool: #
    return text.find("#") >= 0


def read_var(text: str, vs: Dict[str, float], ar: Dict[str, List[float]]) -> float: #
    if not is_var(text):
        return float(text)

    if is_array(text):
        name, index = text.split("#", 1)
        index = int(read_var(index, vs, ar))
        return ar[name][index]
    else:
        return vs[text]


def write_var(text: str, val: float, vs: Dict[str, float], ar: Dict[str, List[float]]): #
    if is_array(text):
        name, index = text.split("#")
        index = int(read_var(index, vs, ar))
        ar[name][index] = val
    else:
        vs[text] = val


def interpret_file(file_path: str):
    with open(file_path, "r", encoding='utf-8') as f:
        lines = f.readlines()
        i: int = 0
        vs: Dict[str, float] = {}
        ar: Dict[str, List[float]] = {}
        while i < len(lines):
            ins = lines[i].replace("\n", "").split(" ")

            if ins[0] == INSTRUCTION_NAMES[InstructionType.READ]:
                v = float(input("> "))
                write_var(ins[1], v, vs, ar)
                i += 1
            elif ins[0] == INSTRUCTION_NAMES[InstructionType.WRITE]:
                v = ins[1]
                if is_var(ins[1]):
                    v = read_var(v, vs, ar)
                print(v)
                i += 1
            elif ins[0] == INSTRUCTION_NAMES[InstructionType.ARRAY]: #
                elems = []
                for a in ins[2:]:
                    if is_var(a):
                        a = read_var(a, vs, ar)
                    elems.append(float(a))
                ar[ins[1]] = elems
                i += 1
            elif ins[0] == INSTRUCTION_NAMES[InstructionType.COPY]:
                a, b = ins[1], ins[2]
                if is_var(a):
                    a = read_var(a, vs, ar)
                write_var(b, float(a), vs, ar)
                i += 1
            elif ins[0] == INSTRUCTION_NAMES[InstructionType.GOTO_IF]:
                v = ins[1]
                if is_var(ins[1]):
                    v = read_var(v, vs, ar)
                if float(v) > 0:
                    i = int(ins[2])
                else:
                    i += 1
            elif ins[0] == INSTRUCTION_NAMES[InstructionType.GOTO_IF_NOT]:
                v = ins[1]
                if is_var(ins[1]):
                    v = read_var(v, vs, ar)
                if not float(v) > 0:
                    i = int(ins[2])
                else:
                    i += 1
            elif ins[0] == INSTRUCTION_NAMES[InstructionType.GOTO]:
                i = int(ins[1])
            elif ins[0] == INSTRUCTION_NAMES[InstructionType.ADD]:
                a, b, c = ins[1], ins[2], ins[3]
                if is_var(a):
                    a = read_var(a, vs, ar)
                if is_var(b):
                    b = read_var(b, vs, ar)
                res = float(a) + float(b)
                write_var(c, res, vs, ar)
                i += 1
            elif ins[0] == INSTRUCTION_NAMES[InstructionType.SUB]:
                a, b, c = ins[1], ins[2], ins[3]
                if is_var(a):
                    a = read_var(a, vs, ar)
                if is_var(b):
                    b = read_var(b, vs, ar)
                res = float(a) - float(b)
                write_var(c, res, vs, ar)
                i += 1
            elif ins[0] == INSTRUCTION_NAMES[InstructionType.MUL]:
                a, b, c = ins[1], ins[2], ins[3]
                if is_var(a):
                    a = read_var(a, vs, ar)
                if is_var(b):
                    b = read_var(b, vs, ar)
                res = float(a) * float(b)
                write_var(c, res, vs, ar)
                i += 1
            elif ins[0] == INSTRUCTION_NAMES[InstructionType.DIV]:
                a, b, c = ins[1], ins[2], ins[3]
                if is_var(a):
                    a = read_var(a, vs, ar)
                if is_var(b):
                    b = read_var(b, vs, ar)
                res = float(a) / float(b)
                write_var(c, res, vs, ar)
                i += 1
            else:
                assert False, f"Not implemented {ins[0]}"


if __name__ == '__main__' and '__file__' in globals():
    argv = sys.argv
    assert len(argv) > 1, "Please provide command & file"
    compiler_name, *argv = argv

    command: str = argv[0]
    file: str = argv[1]

    if command == "com":
        if not file.endswith(".my"):
            compile_error("Invalid file extension, expected .my")

        print(f"Compiling {file} using {compiler_name}...\n")
        inst = parse_program_from_file(file)
        write_instruction(file + "m", inst)
    elif command == "run":
        if not file.endswith(".mym"):
            compile_error("Invalid file extension, expected .mym")

        print(f"Running {file} using {compiler_name}...\n")

        interpret_file(file)
    elif command == "cnr":
        if not file.endswith(".my"):
            compile_error("Invalid file extension, expected .my")

        print(f"Compiling {file} using {compiler_name}...\n")
        inst = parse_program_from_file(file)

        file += "m"
        write_instruction(file, inst)

        print(f"Running {file} using {compiler_name}...\n")

        interpret_file(file)
    else:
        print(f"Unknown command '{command}'")
